In [1]:
import arcpy
import os
from dotenv import load_dotenv
from pathlib import Path

load_dotenv()
gdb_path = os.getenv('GDB_PATH')
# Convert the file path to a raw string
gdb_path = r"{}".format(gdb_path)
feature_class_path = os.path.join(gdb_path,"bad_food")
feature_class_path = r"{}".format(feature_class_path)


grocery_path = os.path.join(gdb_path,"feature_class_2_grocery_v2")
grocery_path = r"{}".format(grocery_path)

In [2]:
### Check that both the geodatabase and feature class exist
if arcpy.Exists(gdb_path): 
    print(f"Geodatabase exists: {gdb_path}")
if arcpy.Exists(feature_class_path):
    print(f"Feature Class exists: {feature_class_path}" )
if arcpy.Exists(grocery_path):
    print(f"Feature Class exists: {grocery_path}" )

Geodatabase exists: C:\Users\jbfre\Documents\ArcGIS\Projects\From_Desert2_Oasis\From_Desert2_Oasis.gdb
Feature Class exists: C:\Users\jbfre\Documents\ArcGIS\Projects\From_Desert2_Oasis\From_Desert2_Oasis.gdb\bad_food
Feature Class exists: C:\Users\jbfre\Documents\ArcGIS\Projects\From_Desert2_Oasis\From_Desert2_Oasis.gdb\feature_class_2_grocery_v2


In [3]:


# Define input and output feature classes
unhealthy_stores = feature_class_path
all_grocery_stores = grocery_path   
buffer_output = os.path.join(gdb_path,"optimal_discov")
spatial_join_output = os.path.join(gdb_path,"join_output")

# Step 1: Create a 2-mile buffer around the unhealthy stores
arcpy.analysis.Buffer(
    in_features=unhealthy_stores,
    out_feature_class=buffer_output,
    buffer_distance_or_field="0.5 Miles"
)

# Step 2: Perform a spatial join to find grocery stores within the buffer
arcpy.analysis.SpatialJoin(
    target_features=buffer_output,
    join_features=all_grocery_stores,
    out_feature_class=spatial_join_output,
    join_type="KEEP_ALL"
)


<Result 'C:\\Users\\jbfre\\Documents\\ArcGIS\\Projects\\From_Desert2_Oasis\\From_Desert2_Oasis.gdb\\join_output'>

In [4]:
# Select buffers with no intersecting grocery stores
no_nearby_stores = os.path.join(gdb_path,"no_nearby_stores")

arcpy.management.SelectLayerByAttribute(
    in_layer_or_view=spatial_join_output,
    where_clause="Join_Count = 0"
)
arcpy.management.CopyFeatures(
    in_features=spatial_join_output, 
    out_feature_class=no_nearby_stores
)


<Result 'C:\\Users\\jbfre\\Documents\\ArcGIS\\Projects\\From_Desert2_Oasis\\From_Desert2_Oasis.gdb\\no_nearby_stores'>

In [7]:


# Create a layer for unhealthy stores
arcpy.management.MakeFeatureLayer(feature_class_path, "UnhealthyStores_Layer_v2")

# Select unhealthy stores that are within 0.5 miles of any grocery store
arcpy.management.SelectLayerByLocation(
    in_layer="UnhealthyStores_Layer_v2",
    overlap_type="WITHIN_A_DISTANCE",
    select_features=all_grocery_stores,
    search_distance="0.5 Miles",
    selection_type="NEW_SELECTION"  # Select features with nearby grocery stores
)

# Step 2: Invert the selection
# Select all features first
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="UnhealthyStores_Layer_v2",
    selection_type="NEW_SELECTION"
)

# Remove the previously selected features
arcpy.management.SelectLayerByLocation(
    in_layer="UnhealthyStores_Layer_v2",
    overlap_type="WITHIN_A_DISTANCE",
    select_features=all_grocery_stores,
    search_distance="0.5 Miles",
    selection_type="REMOVE_FROM_SELECTION"
)

# Save the result to a new feature class
no_nearby_stores_v2 = os.path.join(gdb_path,"no_nearby_stores_v2")
arcpy.management.CopyFeatures("UnhealthyStores_Layer_v2", no_nearby_stores_v2)


<Result 'C:\\Users\\jbfre\\Documents\\ArcGIS\\Projects\\From_Desert2_Oasis\\From_Desert2_Oasis.gdb\\no_nearby_stores_v2'>

In [16]:
feature_class_name = "random_grocery_store"

# Define the geometry type (Point, Polyline, Polygon, etc.)
geometry_type = "POINT"

# Define the spatial reference (e.g., WGS 1984)
spatial_reference = arcpy.SpatialReference(4326)  # WGS 1984

output_workspace = gdb_path
# Create the feature class
arcpy.CreateFeatureclass_management(
    output_workspace,
    feature_class_name,
    geometry_type,
    spatial_reference=spatial_reference
)



# Define output for random points
random_grocery_stores = os.path.join(gdb_path,"random_grocery_store")

# Create random points (1 point per buffer)
arcpy.management.CreateRandomPoints(
    out_path=gdb_path,
    out_name="RandomGroceryStores",
    constraining_feature_class=random_grocery_stores,

)



<Result 'C:\\Users\\jbfre\\Documents\\ArcGIS\\Projects\\From_Desert2_Oasis\\From_Desert2_Oasis.gdb\\RandomGroceryStores'>

In [30]:
import arcpy
import random
import math
import os

# Set the input feature class and the output path for the random points
input_fc = no_nearby_stores_v2  # Path to the feature class with the original points
output_fc_points = os.path.join(gdb_path,"final_out")
  # Path where you want to save the new random points
output_fc_buffers = os.path.join(gdb_path,"final_buf")
  # Path to save the buffers
buffer_distance = 0.5  # Set buffer distance to 0.5 miles

arcpy.management.CreateFeatureclass(gdb_path,output_fc_points,geometry_type="POINT")
arcpy.management.CreateFeatureclass(gdb_path,output_fc_buffers,geometry_type="POLYGON")

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000354: The name contains invalid characters
Failed to execute (CreateFeatureclass).


In [27]:

for field in arcpy.ListFields(output_fc_points):
    print(field.name)

OBJECTID
Shape


In [29]:


# Create an empty list to store the new random points
random_points = []
buffer_distance = 0.5
# Start an edit session if needed for batch processing
with arcpy.da.SearchCursor(input_fc, ['SHAPE@XY']) as cursor:
    for row in cursor:
        # Get the XY coordinates of the point
        x, y = row[0]
        
        # Generate random distances and angles
        radius = 0.5  # radius in miles
        angle = random.uniform(0, 360)  # random angle in degrees
        distance = random.uniform(0, radius)  # random distance within the radius
        
        # Convert the angle to radians
        angle_rad = math.radians(angle)
        
        # Calculate the random point's X and Y coordinates based on polar coordinates
        delta_x = distance * math.cos(angle_rad)
        delta_y = distance * math.sin(angle_rad)
        
        # Add the random point by shifting the original point by the random delta_x and delta_y
        random_x = x + delta_x
        random_y = y + delta_y
        
        # Store the new point in the random_points list
        random_points.append((random_x, random_y))

# Create a new feature class to store the random points

# Insert the random points into the new feature class
with arcpy.da.InsertCursor(output_fc_points, ['SHAPE@XY']) as cursor:
    for point in random_points:
        cursor.insertRow([point])

# Create the buffer around each random point
with arcpy.da.SearchCursor(output_fc_points, ['SHAPE@XY']) as cursor:
    for row in cursor:
        point_geometry = arcpy.PointGeometry(arcpy.Point(row[0][0], row[0][1]))
        
        # Create the buffer around each point
        buffer = point_geometry.buffer(buffer_distance)
        
        # Insert the buffer into the output feature class
        with arcpy.da.InsertCursor(output_fc_buffers, ['SHAPE@']) as insert_cursor:
            insert_cursor.insertRow([buffer])

print("Random points and their buffers generated successfully!")


Random points and their buffers generated successfully!
